```
23일 토요일 각자 eda 후 올리기
24일 일요일 슬랙 또는 행아웃


EDA 해볼 것
1. 사람이 어디서 어디로 가장 많이 이동하는 지. 새로운 노선 만들기(인구수가 많은 지역(유동인구), 정류장 이용 많은 곳)
2. 시간대 이용자 파악 후 버스 배차간격 조정 - 주중, 주말 나눠서 분석 (예측)
3. 신도시 버스 인프라 먼저 파악 (지하철x)
4. 서울에서 오는 버스가 어느 정류장에 서는지 확인

최적화 방법
1. 가장 많은 이용자가 있는 정류장 최단거리
2. 비용을 최소화 하는 방법 (최대한 노선 합치기)
3. 있는 노선 확인 후 사람을 많이 태우는 노선 추천(수요)
```

In [1]:
%matplotlib inline

import geopandas as gpd

import pandas as pd
import os
import requests
import folium
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
# import plotly.express as px
# import plotly.graph_objects as go

In [2]:
# matplotlib에 “AppleGothc”(Mac) 또는 “Malgun Gothic”(Windows) 글꼴 적용

import matplotlib.font_manager as fm

for font in fm.fontManager.ttflist:
    if font.name in ['AppleGothic', 'Malgun Gothic']:
        plt.rcParams['font.family'] = font.name
        break

plt.rcParams['font.family']

['Malgun Gothic']

In [72]:
#2018년 7월 승차일 기준 1~4일의 버스 카드태깅 정보를 담고있습니다.
TripChain = pd.read_csv('./Data/PJT001_TripChain.csv')

#17~18년 기준, 경기도 버스 정류장에 대한 정보를 담고있습니다.
StationTable = pd.read_csv('./Data/PJT001_stations_table.csv')

#2018년 7월 1~4일의 행정동별 이동 인구수 정보를 담고 있습니다.
AreaPeople = pd.read_csv('./Data/PJT001_sk_emd_od.csv')

#버스노선-정류장 매핑 테이블 정보 입니다.
RouteStationInfo = pd.read_csv('./Data/PJT001_routestationinfo.csv')

In [9]:
TripChain.columns=['암호화카드번호', '트랜잭션ID', '환승횟수', '교통카드발행사ID',
       '총이용객수', '사용자구분', '교통수단CD1', '교통수단CD2',
       '교통수단CD3', '교통수단CD4', '교통수단CD5', '버스노선ID1',
       '버스노선ID2', '버스노선ID3', '버스노선ID4', '버스노선ID5',
       '차량ID1', '차량ID2', '차량ID3', '차량ID4',
       '차량ID5', '총통행거리', '총탑승시간', '총소요시간',
       '승차일시1', '승차일시2', '승차일시3', '승차일시4',
       '승차일시5', '하차일시1', '하차일시2', '하차일시3',
       '하차일시4', '하차일시5', '최초승차일시', '최종하차일시',
       '승차역ID1', '승차역ID2', '승차역ID3', '승차역ID4',
       '승차역ID5', '하차역ID1', '하차역ID2', '하차역ID3',
       '하차역ID4', '하차역ID5', '최초승차역ID', '최종하차역ID',
       '총이용금액', '수집건수', '트립체인완료코드']

In [10]:
TripChain.head()

암호화카드번호  트랜잭션ID  환승횟수  교통카드발행사ID  총이용객수  사용자구분  교통수단CD1  교통수단CD2  \
0  900079696430      56     2    9000923      1      1    500.0    500.0   
1  900079697651       5     1    9000923      1      1    500.0      NaN   
2  900079698254      32     1    9000923      1      1    500.0      NaN   
3  900079699257      80     1    9000923      1      1    500.0      NaN   
4  900079701419      64     1    9000923      1      1    530.0      NaN   

   교통수단CD3  교통수단CD4  ...  하차역ID1  하차역ID2  하차역ID3  하차역ID4  하차역ID5  최초승차역ID  \
0      NaN      NaN  ...     NaN     NaN     NaN     NaN     NaN      NaN   
1      NaN      NaN  ...     NaN     NaN     NaN     NaN     NaN      NaN   
2      NaN      NaN  ...     NaN     NaN     NaN     NaN     NaN      NaN   
3      NaN      NaN  ...     NaN     NaN     NaN     NaN     NaN      NaN   
4      NaN      NaN  ...     NaN     NaN     NaN     NaN     NaN      NaN   

     최종하차역ID  총이용금액  수집건수  트립체인완료코드  
0  4116708.0   1350     2         ;  
1  4117269.0   1250     1         ;  
2  4107936.0   1550     1         ;  
3  4116717.0   1250     1         ;  
4  4116848.0   2050     1         ;  

[5 rows x 51 columns]

날짜, 시간 데이터 datetime으로 데이터타입 변경

In [27]:
# TripChain[['승차일시1', '승차일시2']].astype(datetime)



# TripChain['승차일시1'] = pd.to_datetime(TripChain['승차일시1'], format='%Y%m%d%H%S%f')
TripChain['승차일시2'] = pd.to_datetime(TripChain['승차일시2'], format='%Y%m%d%H%S%f')
TripChain['승차일시3'] = pd.to_datetime(TripChain['승차일시3'], format='%Y%m%d%H%S%f')
TripChain['승차일시4'] = pd.to_datetime(TripChain['승차일시4'], format='%Y%m%d%H%S%f')
TripChain['승차일시5'] = pd.to_datetime(TripChain['승차일시5'], format='%Y%m%d%H%S%f')
TripChain['하차일시1'] = pd.to_datetime(TripChain['하차일시1'], format='%Y%m%d%H%S%f')
TripChain['하차일시2'] = pd.to_datetime(TripChain['하차일시2'], format='%Y%m%d%H%S%f')
TripChain['하차일시3'] = pd.to_datetime(TripChain['하차일시3'], format='%Y%m%d%H%S%f')
TripChain['하차일시4'] = pd.to_datetime(TripChain['하차일시4'], format='%Y%m%d%H%S%f')
TripChain['하차일시5'] = pd.to_datetime(TripChain['하차일시5'], format='%Y%m%d%H%S%f')

In [29]:
TripChain.shape

(1048575, 51)

In [30]:
TripChain['암호화카드번호'].nunique()

498764

In [32]:
TripChain['승차일시1'].unique()

array(['2018-07-01T05:00:25.430000000', '2018-07-01T07:00:21.560000000',
       '2018-07-01T12:00:36.530000000', ...,
       '2018-07-04T14:00:02.590000000', '2018-07-04T09:00:49.560000000',
       '2018-07-04T21:00:21.540000000'], dtype='datetime64[ns]')

---

버스노선-정류장 매핑 정보
```
'seq ' : 순번.
'pr_station_id' : 노선 ID.
'bus_line_no' : 버스 노선 번호.
'bus_line_no_seq' : 버스 라인 정류장 순서.
'station_nm ' : 정류장 명칭.
'station_id ' : 표준정류장ID.
'mobile_no' : 모바일정류장ID.
```

In [35]:
# 버스 개수
RouteStationInfo['bus_line_no'].nunique()

226

In [38]:
RouteStationInfo.head()

seq  pr_station_id bus_line_no  bus_line_no_seq station_nm  station_id  \
0  65286      228000018        10-4                1      용인터미널   228001552   
1  65287      228000018        10-4                2  용인터미널(경유)   277102443   
2  65288      228000018        10-4                3      포브스병원   228000443   
3  65289      228000018        10-4                4       제일교회   228000665   
4  65290      228000018        10-4                5     라이프아파트   228000664   

   mobile_no  
0    47634.0  
1        NaN  
2    29439.0  
3    29881.0  
4    29457.0

In [73]:
StationTable.shape

(38509, 12)

In [74]:
StationTable.drop_duplicates(subset ="정류소번호", 
                     keep = 'first', inplace = True) 

In [75]:
StationTable.shape

(31686, 12)

## 버스별로 노선 확인 (Folium 이용)

In [76]:
StationTable.sample()

표준정류장ID  시군명   정류소명          정류소영문명    정류소번호 중앙차로여부     관할관청  \
9432  218000558  고양시  용두사거리  Yongdu Sageori  19535.0  노변정류장  경기도 고양시   

                  위치  WGS84위도   WGS84경도 모바일정류장ID  이비카드정류장ID  
9432  경기도 고양시덕양구 용두동  37.6313  126.8815      NaN        NaN

In [78]:
StationTable[StationTable['표준정류장ID'] == 228001552]

표준정류장ID  시군명   정류소명            정류소영문명    정류소번호 중앙차로여부     관할관청  \
18624  228001552  용인시  용인터미널  Yongin Terminal   47634.0  노변정류장  경기도 용인시   

                     위치    WGS84위도   WGS84경도  \
18624  경기도 용인시 처인구 김량장동  37.232783  127.2101   

                                                모바일정류장ID  이비카드정류장ID  
18624  [None None None None None None None None None ...  4176783.0

In [80]:
RouteStationInfo_latlon = pd.merge(RouteStationInfo, 
             StationTable[['정류소명', '표준정류장ID', 'WGS84위도', 'WGS84경도', '관할관청']],
             left_on='station_id', right_on='표준정류장ID', how='left')

In [81]:
RouteStationInfo_latlon

seq  pr_station_id bus_line_no  bus_line_no_seq       station_nm  \
0       65286      228000018        10-4                1            용인터미널   
1       65287      228000018        10-4                2        용인터미널(경유)   
2       65288      228000018        10-4                3            포브스병원   
3       65289      228000018        10-4                4             제일교회   
4       65290      228000018        10-4                5           라이프아파트   
5       65291      228000018        10-4                6              송담대   
6       65292      228000018        10-4                7  기후변화체험교육센터.삼삼부락   
7       65293      228000018        10-4                8         삼삼부락마을회관   
8       65294      228000018        10-4                9           목동야목농원   
9       65295      228000018        10-4               10           예비군훈련장   
10      65296      228000018        10-4               11               별학   
11      65297      228000018        10-4               12            백마기도원   
12      65298      228000018        10-4               13              장재미   
13      65299      228000018        10-4               14           운학초등학교   
14      65300      228000018        10-4               15           엘에스시스템   
15      65301      228000018        10-4               16               예직   
16      65302      228000018        10-4               17              해곡동   
17      65303      228000018        10-4               18               별미   
18      65304      228000018        10-4               19             와우정사   
19      65305      228000018        10-4               20             곱등고개   
20      65306      228000018        10-4               21               용암   
21      65307      228000018        10-4               22              물구리   
22      65308      228000018        10-4               23            사암주유소   
23      65309      228000018        10-4               24              고당리   
24      65310      228000018        10-4               25               원삼   
25      65311      228000018        10-4               26           원삼초등학교   
26      65312      228000018        10-4               27             독성2리   
27      65313      228000018        10-4               28             두창7리   
28      65314      228000018        10-4               29              두창리   
29      65315      228000018        10-4               30               율곡   
...       ...            ...         ...              ...              ...   
37801  191725      241491011        38-1               38        신한에스빌아파트앞   
37802  191726      241491011        38-1               39          주공1.2단지   
37803  191727      241491011        38-1               40            주공4단지   
37804  191728      241491011        38-1               41          신창2차아파트   
37805  191729      241491011        38-1               42           안화고등학교   
37806  191730      241491011        38-1               43     안화초교.주공8.9단지   
37807  191731      241491011        38-1               44             중심상가   
37808  191732      241491011        38-1               45            롯데시네마   
37809  191733      241491011        38-1               46              진안동   
37810  191734      241491011        38-1               47        안용중학교.현충탑   
37811  191735      241491011        38-1               48            송산동입구   
37812  191736      241491011        38-1               49          안녕농협삼거리   
37813  191737      241491011        38-1               50         남수원현대아파트   
37814  191738      241491011        38-1               51       우남블루존.원광병원   
37815  191739      241491011        38-1               52              용주사   
37816  191740      241491011        38-1               53             안녕자원   
37817  191741      241491011        38-1               54       지킴이마을.타래치길   
37818  191742      241491011        38-1               55              융건릉   
37819  191743      241491011        38-1               56             안녕4통   
37820  191744      241491011    

In [82]:
RouteStationInfo_latlon.isnull().sum()

seq                   0
pr_station_id         0
bus_line_no           0
bus_line_no_seq       0
station_nm            0
station_id            0
mobile_no          2350
정류소명               4627
표준정류장ID            4627
WGS84위도            4627
WGS84경도            4627
관할관청               5538
dtype: int64

총 4627개의 결측값이 존재하는 정류장을 어떻게 해야하나...흠...

In [88]:
# 일단 결측값 데이터 제외
RouteStationInfo_latlon = RouteStationInfo_latlon.dropna()
RouteStationInfo_latlon

seq  pr_station_id bus_line_no  bus_line_no_seq       station_nm  \
0       65286      228000018        10-4                1            용인터미널   
2       65288      228000018        10-4                3            포브스병원   
3       65289      228000018        10-4                4             제일교회   
4       65290      228000018        10-4                5           라이프아파트   
5       65291      228000018        10-4                6              송담대   
6       65292      228000018        10-4                7  기후변화체험교육센터.삼삼부락   
7       65293      228000018        10-4                8         삼삼부락마을회관   
8       65294      228000018        10-4                9           목동야목농원   
9       65295      228000018        10-4               10           예비군훈련장   
10      65296      228000018        10-4               11               별학   
11      65297      228000018        10-4               12            백마기도원   
12      65298      228000018        10-4               13              장재미   
13      65299      228000018        10-4               14           운학초등학교   
14      65300      228000018        10-4               15           엘에스시스템   
15      65301      228000018        10-4               16               예직   
16      65302      228000018        10-4               17              해곡동   
17      65303      228000018        10-4               18               별미   
18      65304      228000018        10-4               19             와우정사   
19      65305      228000018        10-4               20             곱등고개   
20      65306      228000018        10-4               21               용암   
21      65307      228000018        10-4               22              물구리   
22      65308      228000018        10-4               23            사암주유소   
23      65309      228000018        10-4               24              고당리   
24      65310      228000018        10-4               25               원삼   
25      65311      228000018        10-4               26           원삼초등학교   
26      65312      228000018        10-4               27             독성2리   
27      65313      228000018        10-4               28             두창7리   
28      65314      228000018        10-4               29              두창리   
29      65315      228000018        10-4               30               율곡   
30      65316      228000018        10-4               31             창동마을   
...       ...            ...         ...              ...              ...   
37794  191718      241491011        38-1               31         신한에스빌아파트   
37795  191719      241491011        38-1               32           동부출장소앞   
37796  191720      241491011        38-1               33           병점역사거리   
37797  191721      241491011        38-1               34        홈플러스.벌말초교   
37798  191722      241491011        38-1               35      우남퍼스트빌3차아파트   
37799  191723      241491011        38-1               36       신창미션힐.송화초교   
37800  191724      241491011        38-1               37            동부출장소   
37801  191725      241491011        38-1               38        신한에스빌아파트앞   
37802  191726      241491011        38-1               39          주공1.2단지   
37803  191727      241491011        38-1               40            주공4단지   
37804  191728      241491011        38-1               41          신창2차아파트   
37805  191729      241491011        38-1               42           안화고등학교   
37806  191730      241491011        38-1               43     안화초교.주공8.9단지   
37807  191731      241491011        38-1               44             중심상가   
37808  191732      241491011        38-1               45            롯데시네마   
37809  191733      241491011        38-1               46              진안동   
37810  191734      241491011        38-1               47        안용중학교.현충탑   
37811  191735      241491011        38-1               48            송산동입구   
37812  191736      241491011        38-1               49          안녕농협삼거리   
37813  191737      241491011    